# TODO

- Use the preferred terms rather than the first synonyms

In [ ]:
!pip install transformers
!pip install deepl
!pip install tqdm
!pip install evaluate
!pip install termcolor
!pip install Levenshtein
!pip install nltk
!pip install cer
!pip install accelerate

In [1]:
# import os
# os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/cache/'

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from snomed_graph import *
import getpass
import deepl
from tqdm.notebook import tqdm
import json
import numpy as np
import evaluate
from termcolor import colored
from collections import namedtuple
from operator import __or__
from functools import reduce
from ast import literal_eval
from Levenshtein import ratio
from itertools import chain

In [3]:
AYA_CHECKPOINT = "CohereForAI/aya-101"
PATH_TO_SERIALIZED_SNOMED_GRAPH = "./data/snomed_graph/full_concept_graph.gml"
PATH_TO_TRANSLATION_SAMPLES = "./data/prepared_translation_data/samples.csv"
PATH_TO_ALL_TRANSLATION_REFERENCES = "./data/prepared_translation_data/sampling_df.csv"
PATH_TO_DEEPL_TRANSLATION_RESULTS = "./data/cache/deepl_results.json"
PATH_TO_AYA_VANILLA_TRANSLATION_RESULTS = "./data/cache/aya_results_vanilla.json"
PATH_TO_AYA_ENRICHED_TRANSLATION_RESULTS = "./data/cache/aya_results_enriched.json"
ALL_OUTPUT_PATH = "./data/translation_outputs/all_translations.csv"
CT1_OUTPUT_PATH = "./data/translation_outputs/ct1_translations.csv"
CT2_OUTPUT_PATH = "./data/translation_outputs/ct2_translations.csv"

In [4]:
DEEPL_AUTH_KEY = getpass.getpass()

 ········


In [4]:
langcodes = {
    "Dutch": "NL",
    "Estonian": "ET",
    "Korean": "KO",
    "Swedish": "SV",
}

In [5]:
important_attributes = {
    # 'Access (attribute)',
    # 'After (attribute)',
    'Associated finding (attribute)',
    'Associated morphology (attribute)',
    'Associated procedure (attribute)',
    'Associated with (attribute)',
    'Before (attribute)',
    'Causative agent (attribute)',
    'Characterizes (attribute)',
    # 'Clinical course (attribute)',
    'Component (attribute)',
    'Direct device (attribute)',
    'Direct morphology (attribute)',
    'Direct site (attribute)',
    'Direct substance (attribute)',
    'Due to (attribute)',
    'During (attribute)',
    # 'Finding context (attribute)',
    'Finding informer (attribute)',
    'Finding method (attribute)',
    'Finding site (attribute)',
    'Has absorbability (attribute)',
    'Has active ingredient (attribute)',
    'Has basic dose form (attribute)',
    'Has basis of strength substance (attribute)',
    'Has coating material (attribute)',
    'Has compositional material (attribute)',
    'Has concentration strength denominator unit (attribute)',
    'Has concentration strength numerator unit (attribute)',
    'Has device intended site (attribute)',
    'Has disposition (attribute)',
    'Has dose form administration method (attribute)',
    'Has dose form intended site (attribute)',
    'Has dose form release characteristic (attribute)',
    'Has dose form transformation (attribute)',
    'Has filling (attribute)',
    'Has focus (attribute)',
    'Has ingredient qualitative strength (attribute)',
    'Has intent (attribute)',
    # 'Has interpretation (attribute)',
    'Has manufactured dose form (attribute)',
    'Has precise active ingredient (attribute)',
    'Has presentation strength denominator unit (attribute)',
    'Has presentation strength numerator unit (attribute)',
    'Has realization (attribute)',
    'Has specimen (attribute)',
    'Has state of matter (attribute)',
    'Has surface texture (attribute)',
    'Has target population (attribute)',
    'Has unit of presentation (attribute)',
    'Indirect device (attribute)',
    'Indirect morphology (attribute)',
    'Inherent location (attribute)',
    'Inheres in (attribute)',
    'Interprets (attribute)',
    # 'Is a (attribute)',
    'Is modification of (attribute)',
    'Is sterile (attribute)',
    'Laterality (attribute)',
    'Measurement method (attribute)',
    'Method (attribute)',
    'Occurrence (attribute)',
    'Pathological process (attribute)',
    'Plays role (attribute)',
    'Precondition (attribute)',
    'Priority (attribute)',
    'Procedure context (attribute)',
    'Procedure device (attribute)',
    'Procedure morphology (attribute)',
    'Procedure site (attribute)',
    'Procedure site - Direct (attribute)',
    'Procedure site - Indirect (attribute)',
    'Process acts on (attribute)',
    'Process duration (attribute)',
    'Process extends to (attribute)',
    'Process output (attribute)',
    'Property (attribute)',
    'Recipient category (attribute)',
    'Relative to (attribute)',
    'Relative to part of (attribute)',
    'Revision status (attribute)',
    'Route of administration (attribute)',
    # 'Scale type (attribute)',
    # 'Severity (attribute)',
    'Specimen procedure (attribute)',
    'Specimen source identity (attribute)',
    'Specimen source morphology (attribute)',
    'Specimen source topography (attribute)',
    'Specimen substance (attribute)',
    # 'Subject relationship context (attribute)',
    'Surgical approach (attribute)',
    'Technique (attribute)',
    # 'Temporal context (attribute)',
    # 'Temporally related to (attribute)',
    # 'Time aspect (attribute)',
    # 'Units (attribute)',
    'Using access device (attribute)',
    'Using device (attribute)',
    'Using energy (attribute)',
    'Using substance (attribute)'
}

# 1. Load the data

## 1.1 Load the concepts to translate

In [6]:
# Columns are: sctid, fsn, hierarchy, language, context_tier, depth_tier, translations
all_df = (
    pd.read_csv(PATH_TO_TRANSLATION_SAMPLES)
    .set_index(["sctid", "language"])
)

all_df.reference_translations = all_df.reference_translations.apply(literal_eval)

all_df.shape[0]

12640

In [7]:
# all_df = all_df.sample(20)

## 1.2 Load the full set of reference translations

In [8]:
# Columns are: sctid, fsn, hierarchy, language, context_tier, depth_tier, translations
ref_df = (
    pd.read_csv(PATH_TO_ALL_TRANSLATION_REFERENCES)
    .set_index(["sctid", "language"])
)

ref_df.translations = ref_df.translations.apply(literal_eval)

ref_df = ref_df.rename(axis="columns", mapper={"translations": "reference_translations"})

ref_df.shape[0]

651355

## 1.3 Load the SNOMED graph object

In [9]:
G = SnomedGraph.from_serialized(PATH_TO_SERIALIZED_SNOMED_GRAPH)

SNOMED graph has 361179 vertices and 1179749 edges


# 2. Evaluation Harness

In [10]:
google_bleu = evaluate.load("google_bleu")
character = evaluate.load("character")

In [11]:
def exact_match(predictions, references):
    N = len(predictions)
    n = 0
    for p, r in zip(predictions, references):
        if p in r:
            n += 1
    return {'exact_match': float(n)/N}

In [12]:
def levenshtein_ratio(predictions, references):
    ratios = [
        np.max([ratio(p, r) for r in refs])
        for p, refs in zip(predictions, references)
    ]
    return {'levenshtein_ratio': np.mean(ratios)}

In [13]:
def evaluate_translations(row_or_df, target_column):
    if isinstance(row_or_df, pd.DataFrame):
        assert target_column in row_or_df.columns    
        candidates = list(row_or_df.to_dict()[target_column].values())
        references = row_or_df.reference_translations.tolist()
    else:
        candidates = [getattr(row_or_df, target_column)]
        references = [row_or_df.reference_translations]
    results = [
        exact_match(predictions=candidates, references=references),
        levenshtein_ratio(predictions=candidates, references=references),
        google_bleu.compute(predictions=candidates, references=references),
        character.compute(predictions=candidates, references=references),
    ]
    results = reduce(__or__, results, dict())
    return results

# 3. Generate baseline translations with DeepL

In [15]:
translator = deepl.Translator(DEEPL_AUTH_KEY)

def translate_with_deepl(df, G):
    
    with open(PATH_TO_DEEPL_TRANSLATION_RESULTS, "r") as f:
        deepl_results = json.load(f)
    
    for it, row in enumerate(tqdm(df.itertuples(), total=df.shape[0])):
        sctid, language = row.Index
        langcode = langcodes[language]
        source_concept = G.get_concept_details(sctid)
        source_preferred_term = source_concept.fsn.replace(f"({source_concept.hierarchy})", "").strip()
        key = str(sctid) + "_" + language
        try:
            yield deepl_results[key]
        except KeyError:
            deepl_result = translator.translate_text(source_preferred_term, target_lang=langcode)
            deepl_results[key] = deepl_result.text
            yield deepl_result.text
        if it % 100 == 0:
            with open(PATH_TO_DEEPL_TRANSLATION_RESULTS, "w") as f:
                json.dump(deepl_results, f)

    with open(PATH_TO_DEEPL_TRANSLATION_RESULTS, "w") as f:
        json.dump(deepl_results, f)

In [16]:
all_df["deepl_translation"] = list(translate_with_deepl(all_df, G))

  0%|          | 0/12640 [00:00<?, ?it/s]

In [17]:
evaluate_translations(all_df, "deepl_translation")

{'exact_match': 0.08662974683544304,
 'levenshtein_ratio': 0.7370298864393613,
 'google_bleu': 0.2459922409495806,
 'cer_score': 0.381689058931287}

# 4. Generate translations using "vanilla" Aya model.

In [18]:
tokenizer = AutoTokenizer.from_pretrained(AYA_CHECKPOINT)
aya_model = AutoModelForSeq2SeqLM.from_pretrained(AYA_CHECKPOINT, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

model-00009-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00010-of-00011.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [19]:
def aya_postprocessor(result):
    return (
        result
        .replace(tokenizer.eos_token, "")
        .replace(tokenizer.pad_token, "")
        .replace(".", "")
        .strip()
    )

## 4.1 Test Aya with a few translations into English.  Let's see what kind of responses it produces

In [20]:
def test_translate_with_aya(df, G):
    for row in tqdm(df.itertuples(), total=df.shape[0]):
        sctid, language = row.Index
        preferred_term = row.reference_translations[0]
        reference_translations = G.get_concept_details(sctid).synonyms
        # ICL
        try:
            icl_row = next(df[(df.index.get_level_values(0) != sctid) & (df.index.get_level_values(1) == language)].sample(1).itertuples())
        except ValueError:
            pass
        else:
            icl_sctid = icl_row.Index[0]
            icl_preferred_term = icl_row.reference_translations[0]
            icl_reference_translations = G.get_concept_details(icl_sctid).synonyms
            # construct prompt
            prompt_template = 'Translate the following clinical concept into English: "{{PREFERRED_TERM}}". {{TRANSLATED_TERM}}.\n'
            prompt = (
                prompt_template.replace("{{PREFERRED_TERM}}", icl_preferred_term).replace("{{TRANSLATED_TERM}}", icl_reference_translations[0]) +
                prompt_template.replace("{{PREFERRED_TERM}}", preferred_term).replace("{{TRANSLATED_TERM}}.\n", "")
            )
            print(prompt)
            input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
            output = aya_model.generate(input, max_new_tokens=256)
            result = tokenizer.decode(output[0])
            result = aya_postprocessor(result)
            
            print(
                colored("\nSCTID: ", "red", attrs=['bold']),
                sctid,
                colored("\nSource Language: ", "red", attrs=['bold']),
                language,
                colored("\nPreferred Term: ", "red", attrs=['bold']),
                preferred_term,
                colored("\nReference Translations: ", "red", attrs=['bold']),
                reference_translations,
                colored("\nAya Translation: ", "red", attrs=['bold']),            
                result,
                "\n\n",
                # colored("\nAya Scores: ", "red", attrs=['bold']),
                # ", ".join([k+": "+str(v) for k,v in scores.items()]),
            )

In [21]:
test_translate_with_aya(all_df.sample(10), G)

  0%|          | 0/10 [00:00<?, ?it/s]

Translate the following clinical concept into English: "Koolis toimetuleku hindamine". Assessment of performance at school.
Translate the following clinical concept into English: "Endosalpingioos". 

SCTID:  55850004 
Source Language:  Estonian 
Preferred Term:  Endosalpingioos 
Reference Translations:  ['Endosalpingiosis'] 
Aya Translation:  Endosalpingitis 


Translate the following clinical concept into English: "subjectieve refractie met nevelmethode". Fogging.
Translate the following clinical concept into English: "bepalen van cobalamine in plasma". 

SCTID:  407682008 
Source Language:  Dutch 
Preferred Term:  bepalen van cobalamine in plasma 
Reference Translations:  ['Plasma vitamin B12 level'] 
Aya Translation:  Cobalamine 


Translate the following clinical concept into English: "bepalen van cobalamine in plasma". Plasma vitamin B12 level.
Translate the following clinical concept into English: "brevetoxine". 

SCTID:  19950006 
Source Language:  Dutch 
Preferred Term:  brevet

## 4.2 Translate from English into our target languages

In [22]:
def prepare_aya_vanilla_prompt(row, G, df):
    sctid, language = row.Index
    concept = G.get_concept_details(sctid)
    preferred_term = concept.fsn.replace(f"({concept.hierarchy})", "").strip()
    if language == "Swedish":
        return f'Translate the following clinical concept into Swedish: "Pain disorder with psychological factor". smärtsyndrom med psykologisk faktor.\nTranslate the following clinical concept into Swedish: "{preferred_term}". '
    elif language == "Estonian":
        return f'Translate the following clinical concept into Estonian: "Osseous choristoma". Luuline koristoom. \nTranslate the following clinical concept into Estonian: "{preferred_term}". '
    elif language == "Korean":
        return f'Translate the following clinical concept into Korean: "Endoscopic excision of lesion of esophagus". 식도 병변 내시경 절제. \nTranslate the following clinical concept into Korean: "{preferred_term}". '
    elif language == "Dutch":
        return f'Translate the following clinical concept into Dutch: "Open repair of lumbar hernia using biological mesh".  open hernioplastiek van hernia lumbalis met biologisch matje.\nTranslate the following clinical concept into Dutch: "{preferred_term}". '
    else:
        raise ValueError()

In [23]:
def translate_with_aya(df, G, prompt_assembler, ref_df=None, results_filepath=None, rebuild=False, save=False):

    if rebuild:
        results = dict()
    else:
        with open(results_filepath, "r") as f:
            results = json.load(f)
    
    for row in tqdm(df.itertuples(), total=df.shape[0]):
        sctid, language = row.Index
        key = str(sctid) + "_" + language
        try:
            yield results[key]
        except KeyError:
            prompt = prompt_assembler(row, G, ref_df)
            input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
            output = aya_model.generate(input, max_new_tokens=256)
            result = tokenizer.decode(output[0])
            result = aya_postprocessor(result)
            results[key] = result
            yield result

    if save:
        with open(results_filepath, "w") as f:
            json.dump(results, f)

In [24]:
all_df["aya_vanilla_translation"] = list(translate_with_aya(
    all_df, G, prepare_aya_vanilla_prompt, PATH_TO_AYA_VANILLA_TRANSLATION_RESULTS, rebuild=True, save=False
))

  0%|          | 0/12640 [00:00<?, ?it/s]

In [25]:
evaluate_translations(all_df, "aya_vanilla_translation")

{'exact_match': 0.03235759493670886,
 'levenshtein_ratio': 0.608165798751417,
 'google_bleu': 0.1350622406639004,
 'cer_score': 0.5526435448522717}

In [33]:
all_df.to_csv(ALL_OUTPUT_PATH)

# 5. Evaluate Aya with enriched prompt

In [ ]:
enriched_prompt_template = """
You are a medical translation expert.
Your job is to translate formal clinical terms found within the SNOMED Concept Terminology into {{TARGET_LANGUAGE}}.
The concept you need to translate is “{{PREFERRED_TERM}}”.
Here is some information about the concept which may help you:
{{SYNONYMS_FRAGMENT}}
{{HIERARCHY_FRAGMENT}}
{{PARENTS_FRAGMENT}}
{{RELATIONSHIPS_FRAGMENT}}
Now, the translation of “{{PREFERRED_TERM}}” into {{TARGET_LANGUAGE}} is:"""

In [ ]:
def generate_prompt_synonyms_fragment(preferred_term, synonyms):
    if len(synonyms) == 0:
        return ""
    else:
        syn_str = '"' + '" and "'.join(synonyms) + '"'
        return f'In English, synonyms for "{preferred_term}" include: {syn_str}.'

In [ ]:
def generate_prompt_hierarchy_fragment(preferred_term, hierarchy):
    return f'"{preferred_term}" is a {hierarchy}.'

In [ ]:
def generate_prompt_parents_fragment(preferred_term, parents):
    fragment = ""
    for p in parents:
        fragment += f'"{preferred_term}" is a kind of {p.synonyms[0]}.\n'
    return fragment

In [ ]:
def generate_prompt_relationships_fragment(preferred_term, relationship_groups):
    fragment = ""
    for g in relationship_groups:
        for r in g.relationships:
            type = r.type.replace(" (attribute)", "").lower()
            tgt = r.tgt.synonyms[0]
            fragment += f'"{preferred_term}" has {type} {tgt}\n'
    return fragment

In [ ]:
def prepare_aya_enriched_prompt(row, G, df):
    sctid, language = row.Index
    concept = G.get_full_concept(sctid)
    preferred_term = concept.synonyms[0]
    return (
        enriched_prompt_template
        .replace("{{TARGET_LANGUAGE}}", language)
        .replace("{{PREFERRED_TERM}}", preferred_term)
        .replace("{{SYNONYMS_FRAGMENT}}", generate_prompt_synonyms_fragment(preferred_term, concept.synonyms[1:]))
        .replace("{{HIERARCHY_FRAGMENT}}", generate_prompt_hierarchy_fragment(preferred_term, concept.hierarchy))
        .replace("{{PARENTS_FRAGMENT}}", generate_prompt_parents_fragment(preferred_term, concept.parents))
        .replace("{{RELATIONSHIPS_FRAGMENT}}", generate_prompt_relationships_fragment(preferred_term, concept.inferred_relationship_groups))
        .replace("\n\n", "\n")
    )

In [ ]:
print(prepare_aya_enriched_prompt(next(all_df.itertuples()), G, df))

In [ ]:
df["aya_enriched_translation"] = list(translate_with_aya(all_df, G, prepare_aya_enriched_prompt, PATH_TO_AYA_ENRICHED_TRANSLATION_RESULTS, rebuild=True))

In [ ]:
evaluate_translations(all_df, "aya_enriched_translation")

# 6. Translate Context Tier 1 Concepts

In [26]:
def prepare_aya_ct1_prompt(row, G, ref_df):
    sctid, language = row.Index
    concept = G.get_full_concept(sctid)
    preferred_term = concept.fsn.replace(f"({concept.hierarchy})", "").strip()
    parent_concepts = [
        G.get_full_concept(p.sctid) for p in concept.parents
    ]
    parent_data = [
        (
            c.fsn.replace(f"({c.hierarchy})", "").strip(),
            ref_df.loc[(c.sctid, language)].reference_translations[0],
        )
        for c in parent_concepts
    ]
    prompt_fragments = [
        f'Translate the following clinical concept into {language}: "{pt}". {rt}.'
        for pt, rt in parent_data
    ]    
    prompt = '\n'.join(prompt_fragments)
    prompt += f'\nTranslate the following clinical concept into {language}: "{preferred_term}". '
    
    return prompt

In [27]:
ct1_df = all_df[all_df.context_tier.isin(["Tier 1", "Tier 2"])]
ct1_df.shape[0]

8657

In [28]:
ct1_df["aya_ct1_translation"] = list(translate_with_aya(ct1_df, G, prepare_aya_ct1_prompt, ref_df, None, rebuild=True, save=False))

  0%|          | 0/8657 [00:00<?, ?it/s]

/tmp/ipykernel_49020/2817531280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct1_df["aya_ct1_translation"] = list(translate_with_aya(ct1_df, G, prepare_aya_ct1_prompt, ref_df, None, rebuild=True, save=False))


In [29]:
evaluate_translations(ct1_df, "aya_ct1_translation")

{'exact_match': 0.13734550075083748,
 'levenshtein_ratio': 0.728187395794738,
 'google_bleu': 0.319951960733121,
 'cer_score': 0.4275629513922285}

In [45]:
ct1_df.to_csv(CT1_OUTPUT_PATH)

# 7. Translate all Context Tier 2 Concepts

In [30]:
ct2_df = ct1_df[ct1_df.context_tier == "Tier 2"]
ct2_df.shape[0]

3385

In [33]:
def prepare_aya_ct2_prompt(row, G, ref_df):
    sctid, language = row.Index
    concept = G.get_full_concept(sctid)
    preferred_term = concept.fsn.replace(f"({concept.hierarchy})", "").strip()
    parent_concepts = [
        G.get_full_concept(p.sctid) for p in concept.parents
    ]
    parent_data = [
        (
            c.fsn.replace(f"({c.hierarchy})", "").strip(),
            ref_df.loc[(c.sctid, language)].reference_translations[0],
        )
        for c in parent_concepts
    ] 
    related_concepts = [
        G.get_full_concept(r.tgt.sctid)
        for g in concept.inferred_relationship_groups
        for r in g.relationships
        if r.type in important_attributes        
    ]    
    relationship_data = [
        (
            c.fsn.replace(f"({c.hierarchy})", "").strip(),
            ref_df.loc[(c.sctid, language)].reference_translations[0],
        )
        for c in related_concepts
    ]
    prompt_fragments = [
        f'Translate the following clinical concept into {language}: "{pt}". {rt}.'
        for pt, rt in chain(parent_data, relationship_data)
    ]
    prompt = '\n'.join(prompt_fragments)
    prompt += f'\nTranslate the following clinical concept into {language}: "{preferred_term}". '
    
    return prompt

In [36]:
print(prepare_aya_ct2_prompt(next(ct2_df.sample(1).itertuples()), G, ref_df))

Translate the following clinical concept into Swedish: "Structure of component of left knee joint". komponent i vänster knäled.
Translate the following clinical concept into Swedish: "Structure of infrapatellar fat pad". infrapatellar fettkudde, struktur.
Translate the following clinical concept into Swedish: "Structure of soft tissue of left knee region". mjukdelar i vänster knäregion, struktur.
Translate the following clinical concept into Swedish: "Left". vänster.
Translate the following clinical concept into Swedish: "Structure of infrapatellar fat pad of left knee". 


In [37]:
ct2_df["aya_ct2_translation"] = list(translate_with_aya(ct2_df, G, prepare_aya_ct2_prompt, ref_df, None, rebuild=True, save=False))

  0%|          | 0/3385 [00:00<?, ?it/s]

/tmp/ipykernel_49020/2093963965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct2_df["aya_ct2_translation"] = list(translate_with_aya(ct2_df, G, prepare_aya_ct2_prompt, ref_df, None, rebuild=True, save=False))


In [46]:
ct2_df.to_csv(CT2_OUTPUT_PATH)

In [39]:
evaluate_translations(ct2_df, "deepl_translation")

{'exact_match': 0.029542097488921712,
 'levenshtein_ratio': 0.7410458413981232,
 'google_bleu': 0.23576125787880856,
 'cer_score': 0.3833977515011684}

In [40]:
evaluate_translations(ct2_df, "aya_vanilla_translation")

{'exact_match': 0.017134416543574595,
 'levenshtein_ratio': 0.6408576229242886,
 'google_bleu': 0.1414389291689905,
 'cer_score': 0.5252350058012286}

In [41]:
evaluate_translations(ct2_df, "aya_ct1_translation")

{'exact_match': 0.14741506646971936,
 'levenshtein_ratio': 0.7522362039048198,
 'google_bleu': 0.34068437378829003,
 'cer_score': 0.3937319129928592}

In [42]:
evaluate_translations(ct2_df, "aya_ct2_translation")

{'exact_match': 0.19409158050221564,
 'levenshtein_ratio': 0.7677767004169138,
 'google_bleu': 0.36000487745396903,
 'cer_score': 0.3735057730979672}

# 8. Final Evaluations

In [17]:
all_df = pd.read_csv(ALL_OUTPUT_PATH).set_index(["sctid", "language"])
ct1_df = pd.read_csv(CT1_OUTPUT_PATH).set_index(["sctid", "language"])
ct2_df = pd.read_csv(CT2_OUTPUT_PATH).set_index(["sctid", "language"])

# Sampling

In [48]:
for row in ct2_df.sample(10).itertuples():
    sctid, language = row.Index
    preferred_term = G.get_concept_details(sctid).synonyms[0]
    deepl_results = evaluate_translations(row, "deepl_translation")
    vanilla_aya_results = evaluate_translations(row, "aya_vanilla_translation")
    ct1_aya_results = evaluate_translations(row, "aya_ct1_translation")
    ct2_aya_results = evaluate_translations(row, "aya_ct2_translation")
    print(
        colored("\nSCTID: ", "red", attrs=['bold']),
        sctid,
        colored("\nTarget Language: ", "red", attrs=['bold']),
        language,
        colored("\nEnglish Preferred Term: ", "red", attrs=['bold']),
        preferred_term,
        colored("\nReference Translations: ", "red", attrs=['bold']),
        row.reference_translations,
        colored("\nDeepL Translation: ", "red", attrs=['bold']),
        row.deepl_translation,
        colored("\nDeepL Scores: ", "red", attrs=['bold']),
        ", ".join([k+": "+str(v) for k,v in deepl_results.items()]),
        colored("\nVanilla Aya Translation: ", "red", attrs=['bold']),
        row.aya_vanilla_translation,
        colored("\nVanilla Aya Scores: ", "red", attrs=['bold']),
        ", ".join([k+": "+str(v) for k,v in vanilla_aya_results.items()]),        
        colored("\nAya CT1 Translation: ", "red", attrs=['bold']),
        row.aya_ct1_translation,
        colored("\nAya CT1 Scores: ", "red", attrs=['bold']),
        ", ".join([k+": "+str(v) for k,v in ct1_aya_results.items()]),          
        colored("\nAya CT2 Translation: ", "red", attrs=['bold']),
        row.aya_ct2_translation,
        colored("\nAya CT2 Scores: ", "red", attrs=['bold']),
        ", ".join([k+": "+str(v) for k,v in ct2_aya_results.items()]),                  
    )


SCTID:  306258004 
Target Language:  Dutch 
English Preferred Term:  Referral to ITU specialist 
Reference Translations:  ['verwijzen naar intensivist', 'verwijzen naar IC-arts', "verwijzen naar specialist 'intensive care'"] 
DeepL Translation:  Verwijzing naar intensive care specialist 
DeepL Scores:  exact_match: 0.0, levenshtein_ratio: 0.7164179104477613, google_bleu: 0.14285714285714285, cer_score: 0.2804878048780488 
Vanilla Aya Translation:  Verwijzing naar een intensivist 
Vanilla Aya Scores:  exact_match: 0.0, levenshtein_ratio: 0.8421052631578947, google_bleu: 0.2, cer_score: 0.22580645161290322 
Aya CT1 Translation:  verwijzen naar ITU specialist 
Aya CT1 Scores:  exact_match: 0.0, levenshtein_ratio: 0.7272727272727273, google_bleu: 0.3, cer_score: 0.3448275862068966 
Aya CT2 Translation:  verwijzen naar ITU specialist 
Aya CT2 Scores:  exact_match: 0.0, levenshtein_ratio: 0.7272727272727273, google_bleu: 0.3, cer_score: 0.3448275862068966

SCTID:  66498007 
Target Language:

In [19]:
G.get_full_concept(306258004).parents

[306253008 | Referral to doctor (procedure)]

In [20]:
G.get_full_concept(306258004).synonyms

['Referral to ITU specialist', 'Referral to intensive care specialist']

In [26]:
ct2_df.loc[306258004,"Dutch"]

hierarchy                                                          procedure
depth_tier                                                              Deep
context_tier                                                          Tier 2
fsn                        Referral to intensive care specialist (procedure)
reference_translations     ['verwijzen naar intensivist', 'verwijzen naar...
deepl_translation                  Verwijzing naar intensive care specialist
aya_vanilla_translation                      Verwijzing naar een intensivist
aya_ct1_translation                            verwijzen naar ITU specialist
aya_ct2_translation                            verwijzen naar ITU specialist
Name: (306258004, Dutch), dtype: object

In [35]:
row = next(ct2_df[ct2_df.index.get_level_values(0) == 278327000].itertuples())
print(prepare_aya_ct2_prompt(row, G, ref_df))

Translate the following clinical concept into Dutch: "Lumbar spinal nerve root structure". structuur van lumbale zenuwwortel.
Translate the following clinical concept into Dutch: "Structure of posterior lumbosacral spinal cord nerve root". structuur van radix dorsalis nervi spinalis lumbosacralis.
Translate the following clinical concept into Dutch: "Structure of spinal nerve ganglion". ganglion sensorium nervi spinalis.
Translate the following clinical concept into Dutch: "Side". zijde.
Translate the following clinical concept into Dutch: "Structure of lumbar spinal ganglion". 


,,hierarchy,depth_tier,context_tier,fsn,reference_translations,deepl_translation,aya_vanilla_translation,aya_ct1_translation,aya_ct2_translation
sctid,language,,,,,,,,,
306258004,Dutch,procedure,Deep,Tier 2,Referral to intensive care specialist (procedure),"['verwijzen naar intensivist', 'verwijzen naar...",Verwijzing naar intensive care specialist,Verwijzing naar een intensivist,verwijzen naar ITU specialist,verwijzen naar ITU specialist


In [28]:
ct2_df

,,hierarchy,depth_tier,context_tier,fsn,reference_translations,deepl_translation,aya_vanilla_translation,aya_ct1_translation,aya_ct2_translation
sctid,language,,,,,,,,,
1230175003,Dutch,body structure,Deep,Tier 2,Structure of limbus corneae of right eye (body...,"['limbus corneae van OD', 'limbus corneae van ...",Structuur van limbus corneae van rechteroog,Structuur van de cornea van het recht oog,Limbus corneae van het rechter oog,Limbus corneae van het rechter oog
41774009,Dutch,body structure,Deep,Tier 2,Structure of plantar surface of toe (body stru...,"['structuur van plantair oppervlak van teen', ...",Structuur van het plantaire oppervlak van de teen,Structuur van de plantar surface van de teen,voetoppervlak van de teen,voetoppervlak
788701004,Dutch,body structure,Deep,Tier 2,Structure of T11 spinal nerve proper (body str...,"['truncus nervi spinalis T11', 'zenuwstam van ...",Structuur van de eigenlijke T11 spinale zenuw,Skeletspierstructuur van de T11,spinale zenuwwortel van T11,spinale zenuwwortel van T11
181003007,Dutch,body structure,Deep,Tier 2,Entire medial fasciculus of brachial plexus (b...,['gehele fasciculus medialis van plexus brachi...,Gehele mediale fasciculus van plexus brachialis,gehele mediale fasciculus van de brachiale plexus,hele mediale fasciculus van plexus brachialis,gehele mediale fasciculus van plexus brachialis
278327000,Dutch,body structure,Deep,Tier 2,Structure of lumbar spinal ganglion (body stru...,"['structuur van lumbaal spinaal ganglion', 'lu...",Structuur van lumbaal spinaal ganglion,Structuur van het lumbale wervelkolom,lumbale zenuwganglion,lumbale zenuwganglion
...,...,...,...,...,...,...,...,...,...,...
372486003,Swedish,substance,Shallow,Tier 2,Substance with 5-hydroxytryptamine-3-receptor ...,['5-HT3-receptorantagonist'],Substans med verkningsmekanism 5-hydroxytrypta...,Substans med 5-hydroxytryptamine-3-receptor an...,5-HT3-receptorantagonist,5-HT3-receptorantagonist
422751003,Swedish,substance,Shallow,Tier 2,Fusafungine (substance),['fusafungin'],Fusafungine,Fusafungin,Fusafungin,Fusafungin
423259008,Swedish,substance,Shallow,Tier 2,Panitumumab (substance),['panitumumab'],Allmän registrator,Panitumumab,Panitumumab,Panitumumab
